In [179]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import regexp_extract, col
import re

In [180]:
from pyspark.sql import SparkSession
MAX_MEMORY ='8g'
spark = SparkSession.builder.appName("practice_wine").config('spark.driver.memory',MAX_MEMORY).config('spark.executer.memory',MAX_MEMORY).getOrCreate()

In [215]:
file_path = f"file:///{wine_data_path.replace(os.sep,'/')}"
file_path

'file:////home/jovyan/work/start_spark/start_spark/running_spark_data/redwine.csv'

In [182]:
import os
cwd = os.getcwd()
wine_data_path= os.path.join(cwd, 'running_spark_data/redwine.csv')
wine_data_path

'/home/jovyan/work/start_spark/start_spark/running_spark_data/redwine.csv'

In [183]:
wine_df = spark.read.csv(wine_data_path, header=True, inferSchema=True)
wine_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- producer: string (nullable = true)
 |-- nation: string (nullable = true)
 |-- local1: string (nullable = true)
 |-- local2: string (nullable = true)
 |-- local3: string (nullable = true)
 |-- local4: string (nullable = true)
 |-- varieties1: string (nullable = true)
 |-- varieties2: string (nullable = true)
 |-- varieties3: string (nullable = true)
 |-- varieties4: string (nullable = true)
 |-- varieties5: string (nullable = true)
 |-- varieties6: string (nullable = true)
 |-- varieties7: string (nullable = true)
 |-- varieties8: string (nullable = true)
 |-- varieties9: string (nullable = true)
 |-- varieties10: string (nullable = true)
 |-- varieties11: string (nullable = true)
 |-- varieties12: string (nullable = true)
 |-- type: string (nullable = true)
 |-- use: string (nullable = true)
 |-- abv: string (nullable = true)
 |-- degree: string (nullable = true)
 |-- sw

In [184]:
from pyspark.sql.functions import regexp_extract, col

wine_df = wine_df.withColumn('sweet', regexp_extract(col('sweet'), r'\d+', 0).cast('int'))
wine_df = wine_df.withColumn('acidity', regexp_extract(col('acidity'), r'\d+', 0).cast('int'))
wine_df = wine_df.withColumn('body', regexp_extract(col('body'), r'\d+', 0).cast('int'))
wine_df = wine_df.withColumn('tannin', regexp_extract(col('tannin'), r'\d+', 0).cast('int'))

In [185]:
wine_df.show(5)

+---+------+--------------------+------------+------+------------+----------+------+------+------------------+----------+--------------+--------------+--------------+------------+----------+----------+----------+-----------+-----------+-----------+----+-----+-----+------+-----+-------+----+------+------+----+---+
|_c0|    id|                name|    producer|nation|      local1|    local2|local3|local4|        varieties1|varieties2|    varieties3|    varieties4|    varieties5|  varieties6|varieties7|varieties8|varieties9|varieties10|varieties11|varieties12|type|  use|  abv|degree|sweet|acidity|body|tannin| price|year| ml|
+---+------+--------------------+------------+------+------------+----------+------+------+------------------+----------+--------------+--------------+--------------+------------+----------+----------+----------+-----------+-----------+-----------+----+-----+-----+------+-----+-------+----+------+------+----+---+
|  0|137197|              Altair|      Altair| Chile|Ra

In [186]:
print(wine_df.select('varieties1').distinct().show(200))

+--------------------+
|          varieties1|
+--------------------+
|      Black Grenache|
|         Mavrodaphne|
|               Ruche|
|Lambrusco Graspar...|
|          Grand Noir|
|       Malbo Gentile|
|            Pinotage|
|            Brunello|
|           Brachetto|
|         Agiorgitiko|
|            Dolcetto|
|        Petit Verdot|
|            Frappato|
|        Nero d'Avola|
|             Lacrima|
|        Uva di Troia|
|             Pignolo|
|          Piedirosso|
|            Viognier|
|          Chardonnay|
|            Aragonez|
|           Carmenere|
|    Prugnolo Gentile|
|        Black Muscat|
|       Tinta de Toro|
|        Susumaniello|
|          Ciliegiolo|
|     Feteasca Neagra|
|Refosco dal Pedun...|
|   Sangiovese Grosso|
|              Freisa|
|        Moscato Rosa|
|         Tempranillo|
|     Garnatxa Peluda|
|          Dornfelder|
|            Nebbiolo|
|       Grenache Noir|
|            Colorino|
|          Sagrantino|
|              Merlot|
|Muscat Bai

In [187]:
print(wine_df.select('varieties2').distinct().count())

139


In [188]:
from pyspark.sql.functions import col

# PySpark DataFrame에서 'varieties1' 컬럼의 빈도수를 구하는 방법
variety_counts_df = wine_df.groupBy('varieties1').count()

# 결과 확인 (count() 컬럼을 기준으로 내림차순 정렬)
variety_counts_df.orderBy(col('count').desc()).show(200)

+--------------------+-----+
|          varieties1|count|
+--------------------+-----+
|  Cabernet Sauvignon| 2784|
|          Pinot Noir| 2258|
|              Merlot| 1656|
|      Syrah & Shiraz| 1338|
|          Sangiovese|  883|
|         Tempranillo|  597|
|            Grenache|  487|
|            Nebbiolo|  444|
|              Malbec|  313|
|               Blend|  304|
|           Carmenere|  252|
|                NULL|  189|
|             Barbera|  175|
|           Zinfandel|  151|
|             Corvina|  146|
|       Montepulciano|  143|
|          Monastrell|  133|
|            Garnacha|  119|
|      Cabernet Franc|  118|
|        Nero d'Avola|   91|
|           Primitivo|   90|
|               Gamay|   87|
|            Carignan|   66|
|   Sangiovese Grosso|   51|
|           Aglianico|   51|
|            Dolcetto|   48|
|  Bordeaux Blend Red|   43|
|          Negroamaro|   42|
|    Corvina Veronese|   39|
|        Petite Sirah|   39|
|   Nerello Mascalese|   37|
|            P

In [189]:
from pyspark.sql.functions import col

# PySpark DataFrame에서 'varieties1' 컬럼의 빈도수를 구하는 방법
variety_counts_df = wine_df.groupBy('local1').count()

# 결과 확인 (count() 컬럼을 기준으로 내림차순 정렬)
variety_counts_df.orderBy(col('count').desc()).show(2000)

+--------------------+-----+
|              local1|count|
+--------------------+-----+
|           Bourgogne| 1629|
|          California| 1620|
|            Bordeaux| 1333|
|             Toscana| 1103|
|     South Australia|  709|
|            Piemonte|  706|
|               Rhone|  544|
|        Rapel Valley|  482|
|     Southern France|  459|
|                NULL|  428|
|             Mendoza|  353|
|        Maipo Valley|  307|
|      Central Valley|  300|
|               Rioja|  268|
|              Veneto|  242|
|             Sicilia|  184|
|    Washington State|  166|
|              Puglia|  155|
|              Oregon|  134|
|        Maule Valley|  130|
|  Cabernet Sauvignon|  129|
|             Abruzzo|  120|
|           Aconcagua|  113|
|      Coastal Region|  108|
|    Ribera del Duero|  102|
|            Victoria|   89|
|           La Mancha|   89|
|        South Island|   84|
|South Eastern Aus...|   69|
|       Syrah  Shiraz|   66|
|       Curico Valley|   63|
|            C

In [190]:
from pyspark.sql.functions import col, sum, desc, expr
from functools import reduce

def filter_by_top_n(df, limit_local1, limit_variety):
    """
    지정된 상위 N개 생산지와 품종으로 데이터프레임을 필터링하는 함수.

    :param df: PySpark DataFrame (wine_df)
    :param limit_local1: 상위 생산지 개수
    :param limit_variety: 상위 품종 개수
    :return: 필터링된 DataFrame과 그 개수
    """
    
    print(f"\n--- 생산지 상위 {limit_local1}개, 품종 상위 {limit_variety}개로 필터링 시작 ---")
    
    # 1. 생산지(local1) 상위 목록 추출
    top_local1_list = [
        row.local1 for row in df.groupBy("local1")
                                 .agg(sum("ml").alias("total_ml"))
                                 .orderBy(desc("total_ml"))
                                 .limit(limit_local1)
                                 .collect()
    ]
    
    # 2. 품종(varieties1 ~ varieties12) 상위 목록 추출
    varieties_cols = [f"varieties{i}" for i in range(1, 13)]
    stack_expr = expr("stack(12, " + ", ".join(f"'{c}', `{c}`" for c in varieties_cols) + ") as (col_name, variety)")
    
    top_variety_df = (
        df.select(col("ml"), stack_expr)
        .groupBy("variety")
        .agg(sum("ml").alias("total_ml"))
        .orderBy(desc("total_ml"))
        .filter(col("variety").isNotNull())
        .limit(limit_variety)
    )
    top_variety_list = [row.variety for row in top_variety_df.collect()]
    
    # 3. 필터링 실행
    variety_filter_condition = reduce(
        lambda x, y: x | y, 
        [col(variety).isin(top_variety_list) for variety in varieties_cols]
    )
    filtered_df = df.filter(
        col("local1").isin(top_local1_list) & 
        variety_filter_condition
    )
    
    # 4. 결과 반환
    filtered_count = filtered_df.count()
    return filtered_df, filtered_count

# 사용 예시
# 데이터프레임 'wine_df'는 미리 선언되어 있어야 합니다.



# 예시 3: 생산지 상위 30개, 품종 상위 25개
filtered_df, count_27_30 = filter_by_top_n(wine_df, 27, 30)
print(f"생산지 20개, 품종 40개로 필터링된 데이터 수: {count_27_30}개")


--- 생산지 상위 27개, 품종 상위 30개로 필터링 시작 ---
생산지 20개, 품종 40개로 필터링된 데이터 수: 11129개


In [191]:
wine_df=filtered_df

In [192]:
wine_df.count()

11129

In [193]:
# price가 0인 와인만 필터링하여 남깁니다.
zero_price_wines_df = wine_df.filter(col("price") == 0)

# 결과 확인
print(f"가격이 0인 와인의 총 개수: {zero_price_wines_df.count()}개")
zero_price_wines_df.show(5)

가격이 0인 와인의 총 개수: 4637개
+---+------+--------------------+-----------------+------+----------+------------------+------+------+------------------+----------+--------------+------------+----------+----------+----------+----------+----------+-----------+-----------+-----------+----+-----+-----+------+-----+-------+----+------+-----+----+---+
|_c0|    id|                name|         producer|nation|    local1|            local2|local3|local4|        varieties1|varieties2|    varieties3|  varieties4|varieties5|varieties6|varieties7|varieties8|varieties9|varieties10|varieties11|varieties12|type|  use|  abv|degree|sweet|acidity|body|tannin|price|year| ml|
+---+------+--------------------+-----------------+------+----------+------------------+------+------+------------------+----------+--------------+------------+----------+----------+----------+----------+----------+-----------+-----------+-----------+----+-----+-----+------+-----+-------+----+------+-----+----+---+
|  4|137201|Benziger, Cabe

In [194]:
from pyspark.sql.functions import col

# 1. 가격이 0보다 큰 와인(6492개)만 남기기
clean_df_step1 = wine_df.filter(col("price") > 0)
print(f"가격이 0보다 큰 와인 개수: {clean_df_step1.count()}개")

# 2. 나머지 조건 적용 (NULL 값 제거)
# abv, local1, varieties1 중 하나라도 NULL인 행들을 제거합니다.
final_clean_df = clean_df_step1.dropna(subset=["abv", "local1", "varieties1"])

# 3. 최종 결과 확인
final_count = final_clean_df.count()
print(f"모든 조건 적용 후 최종 데이터 수: {final_count}개")

final_clean_df.printSchema()

가격이 0보다 큰 와인 개수: 6492개
모든 조건 적용 후 최종 데이터 수: 4367개
root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- producer: string (nullable = true)
 |-- nation: string (nullable = true)
 |-- local1: string (nullable = true)
 |-- local2: string (nullable = true)
 |-- local3: string (nullable = true)
 |-- local4: string (nullable = true)
 |-- varieties1: string (nullable = true)
 |-- varieties2: string (nullable = true)
 |-- varieties3: string (nullable = true)
 |-- varieties4: string (nullable = true)
 |-- varieties5: string (nullable = true)
 |-- varieties6: string (nullable = true)
 |-- varieties7: string (nullable = true)
 |-- varieties8: string (nullable = true)
 |-- varieties9: string (nullable = true)
 |-- varieties10: string (nullable = true)
 |-- varieties11: string (nullable = true)
 |-- varieties12: string (nullable = true)
 |-- type: string (nullable = true)
 |-- use: string (nullable = true)
 |-- abv: string (nullable = t

In [195]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import FloatType
import re

# UDF(사용자 정의 함수) 생성
# '12~13' 형태의 문자열을 받아 평균값(12.5)을 반환하는 함수입니다.
@udf(FloatType())
def calculate_average_abv(abv_str):
    if abv_str is None:
        return None
    
    # 숫자만 추출하기 위해 ~ 기호로 분리
    parts = abv_str.split('~')
    
    try:
        if len(parts) == 2:
            # '12~13'인 경우: (12+13)/2
            return (float(parts[0]) + float(parts[1])) / 2
        elif len(parts) == 1:
            # '12'인 경우: 12
            return float(parts[0])
        else:
            # 예상치 못한 형식인 경우: None 반환
            return None
    except (ValueError, IndexError):
        # 숫자로 변환할 수 없는 경우 None 반환
        return None

# 'final_clean_df'에 새로운 'numeric_abv' 컬럼 추가
# 기존 abv 컬럼은 유지하고, 변환된 값을 새로운 컬럼에 저장합니다.
final_clean_df_with_numeric_abv = final_clean_df.withColumn(
    "numeric_abv", 
    calculate_average_abv(col("abv"))
)

# 결과 확인 (원하는 만큼의 행을 보여줍니다)
final_clean_df_with_numeric_abv.select("abv", "numeric_abv").show(10)
final_clean_df_with_numeric_abv.printSchema()

+-----+-----------+
|  abv|numeric_abv|
+-----+-----------+
|14~15|       14.5|
|14~15|       14.5|
|13~14|       13.5|
|13~14|       13.5|
|13~14|       13.5|
|12~13|       12.5|
|12~13|       12.5|
| 13.5|       13.5|
|13~14|       13.5|
|13~14|       13.5|
+-----+-----------+
only showing top 10 rows

root
 |-- _c0: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- producer: string (nullable = true)
 |-- nation: string (nullable = true)
 |-- local1: string (nullable = true)
 |-- local2: string (nullable = true)
 |-- local3: string (nullable = true)
 |-- local4: string (nullable = true)
 |-- varieties1: string (nullable = true)
 |-- varieties2: string (nullable = true)
 |-- varieties3: string (nullable = true)
 |-- varieties4: string (nullable = true)
 |-- varieties5: string (nullable = true)
 |-- varieties6: string (nullable = true)
 |-- varieties7: string (nullable = true)
 |-- varieties8: string (nullable = true)
 |-- varieties9:

In [196]:
final_clean_df_with_numeric_abv.show(10)

+---+------+--------------------+--------------------+------+---------------+--------------------+------+------+------------------+------------------+--------------+--------------+--------------+------------+----------+----------+----------+-----------+-----------+-----------+----+-----+-----+------+-----+-------+----+------+------+----+---+-----------+
|_c0|    id|                name|            producer|nation|         local1|              local2|local3|local4|        varieties1|        varieties2|    varieties3|    varieties4|    varieties5|  varieties6|varieties7|varieties8|varieties9|varieties10|varieties11|varieties12|type|  use|  abv|degree|sweet|acidity|body|tannin| price|year| ml|numeric_abv|
+---+------+--------------------+--------------------+------+---------------+--------------------+------+------+------------------+------------------+--------------+--------------+--------------+------------+----------+----------+----------+-----------+-----------+-----------+----+-----+

In [197]:
# 'collect()'를 사용하지 않고 변수에 바로 할당
wine_df = final_clean_df_with_numeric_abv

# 이제 wine_df를 사용하면 numeric_abv 컬럼이 포함된 데이터프레임으로 작업할 수 있습니다.
wine_df.show(5)
wine_df.printSchema()

+---+------+--------------------+--------+------+---------------+--------------------+------+------+------------------+------------------+----------+--------------+--------------+------------+----------+----------+----------+-----------+-----------+-----------+----+-----+-----+------+-----+-------+----+------+------+----+---+-----------+
|_c0|    id|                name|producer|nation|         local1|              local2|local3|local4|        varieties1|        varieties2|varieties3|    varieties4|    varieties5|  varieties6|varieties7|varieties8|varieties9|varieties10|varieties11|varieties12|type|  use|  abv|degree|sweet|acidity|body|tannin| price|year| ml|numeric_abv|
+---+------+--------------------+--------+------+---------------+--------------------+------+------+------------------+------------------+----------+--------------+--------------+------------+----------+----------+----------+-----------+-----------+-----------+----+-----+-----+------+-----+-------+----+------+------+--

In [198]:
wine_df.createOrReplaceTempView('wines')

In [199]:
query = """
SELECT 
    id,
    nation,
    local1,
    varieties1,
    numeric_abv,
    sweet,
    acidity,
    body,
    tannin,
    year,
    price
FROM
    wines

"""

In [200]:
wine_df.count()

4367

In [201]:
wine_df = spark.sql(query)

In [202]:
wine_df.show()

+------+------+---------------+------------------+-----------+-----+-------+----+------+----+------+
|    id|nation|         local1|        varieties1|numeric_abv|sweet|acidity|body|tannin|year| price|
+------+------+---------------+------------------+-----------+-----+-------+----+------+----+------+
|137197| Chile|   Rapel Valley|Cabernet Sauvignon|       14.5|    1|      4|   5|     4|2014|220000|
|137198| Chile|   Rapel Valley|Cabernet Sauvignon|       14.5|    1|      3|   4|     4|2016|110000|
|137204|France|       Bordeaux|            Merlot|       13.5|    1|      3|   2|     3|2011| 21000|
|137207|France|Southern France|Cabernet Sauvignon|       13.5|    1|      3|   2|     3|2017| 19000|
|137208|France|Southern France|            Merlot|       13.5|    1|      3|   3|     3|2011| 19000|
|137212|France|       Bordeaux|Cabernet Sauvignon|       12.5|    1|      4|   3|     3|2015| 32000|
|137214|France|       Bordeaux|            Merlot|       12.5|    1|      3|   2|     2|201

In [203]:
stages = []

In [204]:
train_df, test_df = wine_df.randomSplit([0.8,0.2], seed=1)

In [218]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

cat_features = ['local1', 'varieties1', 'nation']
num_features = ['numeric_abv', 'sweet', 'acidity', 'body', 'tannin','year']
stages = []

# 1. 범주형 특성 전처리: StringIndexer -> OneHotEncoder
for cat in cat_features:
    cat_index = StringIndexer(inputCol=cat, outputCol=cat + '_idx').setHandleInvalid('keep')
    onehot_encode = OneHotEncoder(inputCols=[cat_index.getOutputCol()], outputCols=[cat + '_onehot'])
    stages += [cat_index, onehot_encode]

# 2. 숫자형 특성 전처리: 모든 숫자 컬럼을 하나의 벡터로 통합 후 표준화
num_assembler = VectorAssembler(inputCols=num_features, outputCol="num_features_vector", handleInvalid="keep" )
num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol="scaled_num_features")
stages += [num_assembler, num_scaler]

# 3. 최종 통합: 범주형 원-핫 벡터 + 표준화된 숫자형 벡터를 하나의 'feature_vector'로 합치기
assembler_inputs = [cat + '_onehot' for cat in cat_features] + ['scaled_num_features']
final_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol='feature_vector')
stages.append(final_assembler)

# 4. 파이프라인 구축
pipeline = Pipeline(stages=stages)

# 'train_df' 데이터프레임으로 파이프라인 학습 및 변환
fitted_pipeline = pipeline.fit(train_df)
vtrain_df = fitted_pipeline.transform(train_df)

# 결과 확인
vtrain_df.select('feature_vector').show(5, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|feature_vector                                                                                                                                              |
+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(84,[6,26,70,78,79,80,81,82,83],[1.0,1.0,1.0,17.058063566281497,3.839340335335748,6.674252843541482,6.230143632584154,5.1669935146561095,6.745358874762308])|
|(84,[6,26,70,78,79,80,81,82,83],[1.0,1.0,1.0,17.058063566281497,3.839340335335748,5.005689632656112,4.984114906067323,5.1669935146561095,6.752057344349957])|
|(84,[1,28,67,78,79,80,81,82,83],[1.0,1.0,1.0,15.881645389296567,3.839340335335748,5.005689632656112,2.4920574530336617,3.875245135992082,6.735311170380835])|
|(84,[8,26,67,78,79,80,81,82,83],[1.0,1.0,1.0,

In [219]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=stages)
fitted_transform = pipeline.fit(train_df)
vtrain_df = fitted_transform.transform(train_df)
vtrain_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- nation: string (nullable = true)
 |-- local1: string (nullable = true)
 |-- varieties1: string (nullable = true)
 |-- numeric_abv: float (nullable = true)
 |-- sweet: integer (nullable = true)
 |-- acidity: integer (nullable = true)
 |-- body: integer (nullable = true)
 |-- tannin: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- local_varieties: string (nullable = true)
 |-- local1_idx: double (nullable = false)
 |-- local1_onehot: vector (nullable = true)
 |-- varieties1_idx: double (nullable = false)
 |-- varieties1_onehot: vector (nullable = true)
 |-- nation_idx: double (nullable = false)
 |-- nation_onehot: vector (nullable = true)
 |-- num_features_vector: vector (nullable = true)
 |-- scaled_num_features: vector (nullable = true)
 |-- feature_vector: vector (nullable = true)



In [220]:
vtrain_df.select('feature_vector').show()

+--------------------+
|      feature_vector|
+--------------------+
|(84,[6,26,70,78,7...|
|(84,[6,26,70,78,7...|
|(84,[1,28,67,78,7...|
|(84,[8,26,67,78,7...|
|(84,[1,26,67,78,7...|
|(84,[1,28,67,78,7...|
|(84,[1,28,67,78,7...|
|(84,[1,28,67,78,7...|
|(84,[1,26,67,78,7...|
|(84,[1,26,67,78,7...|
|(84,[1,26,67,78,7...|
|(84,[1,26,67,78,7...|
|(84,[10,26,70,78,...|
|(84,[12,26,70,78,...|
|(84,[6,35,70,78,7...|
|(84,[0,41,69,78,7...|
|(84,[0,26,69,78,7...|
|(84,[0,27,69,78,7...|
|(84,[0,26,69,78,7...|
|(84,[0,28,69,78,7...|
+--------------------+
only showing top 20 rows



In [221]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression( maxIter=50, solver='normal', labelCol='price', featuresCol='feature_vector')

In [222]:
model = lr.fit(vtrain_df)

In [223]:
#테스트데이터 변환
vtest_df = fitted_transform.transform(test_df)
#테스트데이터로 예측
pred=model.transform(vtest_df)

In [224]:
pred.cache()

DataFrame[id: int, nation: string, local1: string, varieties1: string, numeric_abv: float, sweet: int, acidity: int, body: int, tannin: int, year: int, price: int, local1_idx: double, local1_onehot: vector, varieties1_idx: double, varieties1_onehot: vector, nation_idx: double, nation_onehot: vector, num_features_vector: vector, scaled_num_features: vector, feature_vector: vector, prediction: double]

In [228]:
pred.select('price','prediction').show(10)

+------+-------------------+
| price|         prediction|
+------+-------------------+
| 19000|  26655.08606853406|
|120000| 287005.82573237584|
|130000| 202416.83717342129|
|450000| 281359.63377355377|
| 22000|  9584.092164004222|
| 34000| 23091.305674985284|
| 98000| 217550.82882237027|
| 45000|  44996.66325966013|
| 19000| -23150.93588494358|
| 17000|-1405.9219292299822|
+------+-------------------+
only showing top 10 rows



In [226]:
model.summary.r2, model.summary.rootMeanSquaredError

(0.2435321302476542, 238084.72148323504)

In [225]:
spark.stop()